# Install Necessary Library

In [ ]:
!pip install ultralytics --no-deps

# Import Necessary Libraries

In [ ]:
from ultralytics import YOLO
import shutil
import os

In [ ]:
!ls /kaggle/input/football-player-detection-yolov8/video.mp4

In [ ]:
!ls /kaggle/input/football-player-detection-yolov8/football_players_detection/football_players_detection

# Implement YOLO detection(yolov8m) to detect objects in images
### Loading Model and Training

In [ ]:

os.environ["WANDB_MODE"] = "disabled"
# Load Model
model = YOLO('yolov8m.pt')

# Train Model
results = model.train(
    data='/kaggle/input/football-player-detection-yolov8/football_players_detection/football_players_detection/data.yaml',
    epochs=30,              
    imgsz=512,             
    batch=8,                
    workers=2,             
    device=0,               
    project='YOLO_Training', 
    name='football_yolo_last-updated', 
    save=True,              
    save_period=10,         
    resume=False,           
    cos_lr=True,
    patience=10,
    verbose=True
)



# Fine Tuning From last Epoch in model yolov8m ,file(last.pt)
### Loading Model from my trained model file(last.pt) and Training


In [ ]:
model = YOLO('YOLO_Training/football_yolo_last-updated/weights/last.pt')
results = model.train(
    data='/kaggle/input/football-player-detection-yolov8/football_players_detection/football_players_detection/data.yaml',
    epochs=30,           
    imgsz=512,
    batch=8,
    workers=2,
    device=0,
    project='YOLO_Training',
    name='football_yolo_finetune_from_last',  
    save=True,
    save_period=5,
    resume=False,        
    freeze=None,         
    lr0=0.0005,
    cos_lr=True,
    patience=10,
    verbose=True
)


# Saving two Model (Basic Model , Fine Tuning Model) in zip file to dowload on my local device

In [ ]:
!zip -r yolo_training_files.zip /kaggle/working/YOLO_Training/

In [ ]:
from IPython.display import FileLink
FileLink('yolo_training_files.zip')

# (Ball) Detection Only (filter to select ball images) and Create Yaml file to these images

In [ ]:
import os
import shutil

old_data_path = "/kaggle/input/football-player-detection-yolov8/football_players_detection/football_players_detection"
new_data_path = "/kaggle/working/datasets/football_ball_only"

ball_class_id = 0

for split in ['train', 'valid', 'test']: 
    os.makedirs(f"{new_data_path}/images/{split}", exist_ok=True)
    os.makedirs(f"{new_data_path}/labels/{split}", exist_ok=True)

def find_image_file(img_dir, base_name):
    for ext in ('.jpg', '.jpeg', '.png'):
        p = os.path.join(img_dir, base_name + ext)
        if os.path.exists(p):
            return p
    return None

summary = {'copied_images': 0, 'copied_labels': 0, 'skipped_labels_no_ball': 0, 'missing_images': 0}
for split in ['train', 'valid', 'test']:
    old_images_dir = os.path.join(old_data_path, split, "images")
    old_labels_dir = os.path.join(old_data_path, split, "labels")
    new_images_dir = os.path.join(new_data_path, "images", split)
    new_labels_dir = os.path.join(new_data_path, "labels", split)
    if not os.path.isdir(old_labels_dir):
        print(f"labels dir not found for split '{split}': {old_labels_dir}  -> skipped")
        continue

    for label_file in sorted(os.listdir(old_labels_dir)):
        if not label_file.endswith(".txt"):
            continue
        old_label_path = os.path.join(old_labels_dir, label_file)
        with open(old_label_path, 'r') as f:
            lines = [ln.rstrip() for ln in f.readlines() if ln.strip()]

        ball_lines = [ln + "\n" for ln in lines if ln.split()[0] == str(ball_class_id)]

        if not ball_lines:
            summary['skipped_labels_no_ball'] += 1
            continue  
        base_name = label_file[:-4]  
        img_path = find_image_file(old_images_dir, base_name)
        if img_path is None:
            summary['missing_images'] += 1
            print(f" Image missing for label: {label_file} (expected in {old_images_dir})")
            with open(os.path.join(new_labels_dir, label_file), 'w') as nf:
                nf.writelines(ball_lines)
            summary['copied_labels'] += 1
            continue

    
        shutil.copy(img_path, os.path.join(new_images_dir, os.path.basename(img_path)))
        with open(os.path.join(new_labels_dir, label_file), 'w') as nf:
            nf.writelines(ball_lines)

        summary['copied_images'] += 1
        summary['copied_labels'] += 1

print("\n Data Ball Only:", new_data_path)
print("Summary:")
for k, v in summary.items():
    print(f"  - {k}: {v}")


# Saving Ball images -> zip file

In [ ]:
!zip -r football_ball_only.zip /kaggle/working/datasets/football_ball_only


In [ ]:
from IPython.display import FileLink
FileLink('football_ball_only.zip')


# Showing in yaml file

In [ ]:
!cat /kaggle/input/football-ball-only-fixed/football_ball_only_fixed.yaml


# Using YOLO detection ( yolo11l) for model(ball images)

In [ ]:
from ultralytics import YOLO

# Load Model
model = YOLO("yolo11l.pt")

# Train Model
results = model.train(
    data="/kaggle/input/football-ball-only-fixed/football_ball_only_fixed.yaml",
    epochs=40,
    imgsz=512,
    batch=8,
    workers=1,
    device=0,
    project="YOLO_Training",
    name="football_ball_only_11l",
    save=True,
    save_period=10,
    cos_lr=True,
    patience=10,
    verbose=True
)


# Fine Tuning to last.pt for model Ball only

In [ ]:
model = YOLO('/kaggle/working/YOLO_Training/football_ball_only_11l4/weights/last.pt')
results = model.train(
    data='/kaggle/input/football-ball-only-fixed/football_ball_only_fixed.yaml',
    epochs=50,           
    imgsz=512,
    batch=8,
    workers=2,
    device=0,
    project='YOLO_Training',
    name='football_ball_only_yolo_finetune_from_best',  
    save=True,
    save_period=10,
    resume=False,        
    freeze=None,         
    lr0=0.0003,
    cos_lr=True,
    patience=10,
    verbose=True
)


# Using Yolo11l on Basic data

In [ ]:
os.environ["WANDB_MODE"] = "disabled"
# Load Model
model = YOLO('yolo11l.pt')

# Train Model
results = model.train(
    data='/kaggle/input/football-player-detection-yolov8/football_players_detection/football_players_detection/data.yaml',
    epochs=30,              
    imgsz=512,             
    batch=8,                
    workers=2,             
    device=0,               
    project='YOLO_Training', 
    name='football_yolo_11l', 
    save=True,              
    save_period=10,         
    resume=False,           
    cos_lr=True,
    patience=10,
    verbose=True
)

